# Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle
import keras
import warnings
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# warnings.filterwarnings('ignore')
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Using TensorFlow backend.


# Load the Train Test Split

In [2]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [3]:
_, num_timeframes, num_features = X_train.shape

# Reshape data for model

In [4]:
X_train = X_train.reshape((-1,num_timeframes,num_features,1))
X_test = X_test.reshape((-1,num_timeframes,num_features,1))
Y_train = keras.utils.to_categorical(Y_train, 6)
Y_test = keras.utils.to_categorical(Y_test, 6)
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12, 1) 
Testing Samples : (83, 250, 12, 1)
Training Labels : (332, 6) 
Testing Labels : (83, 6)


# CNN Model

In [5]:
input_shape = X_train[0].shape
print(input_shape)
batch_size = 5
epochs = 500

model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5),
                 activation='tanh',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(12, (3, 3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(12, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))
print(model.summary())

(250, 12, 1)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 246, 8, 6)         156       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 8, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 121, 6, 12)        660       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 60, 6, 12)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 6, 12)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4320)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)  

In [6]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_data=(X_test, Y_test))

In [7]:
print(history.history.keys())

# Accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

# Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [8]:
score = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [9]:
# # weights = model.get_weights()
# # # pickle.dump(weights, open(os.path.join("..", "IPD", "CNN_model_half.pkl"),"wb"))
# # # print("Model Saved")
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")

In [10]:
weights = model.get_weights()
# pickle.dump(weights, open(os.path.join("..", "IPD", "CNN_model_half.pkl"),"wb"))
# print("Model Saved")
model_json = model.to_json()
with open(os.path.join("..", "IPD", "CNN_model_half_json.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join("..", "IPD", "CNN_model_half_weights.h5"))
# model.save("C:\\Users\\sachi\\Documents\\sem2\\model_final.h5")
print("Saved model to disk")

# Train on entire data

In [11]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))

X_train = np.concatenate((X_train,X_test), axis=0)
Y_train = np.concatenate((Y_train,Y_test), axis=0)

_ , num_timeframes, num_features = X_train.shape
X_train = X_train.reshape((-1,num_timeframes*num_features))
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X_train)

from sklearn.utils import shuffle
X_train, X_sparse, Y_train = shuffle(X_train, X_sparse, Y_train)
X_train = X_train.reshape((-1,num_timeframes,num_features))

In [12]:
X_train = X_train.reshape((-1,num_timeframes,num_features,1))
X_test = X_test.reshape((-1,num_timeframes,num_features,1))

Y_train = keras.utils.to_categorical(Y_train, 6)
Y_test = keras.utils.to_categorical(Y_test, 6)
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (415, 250, 12, 1) 
Testing Samples : (83, 250, 12, 1)
Training Labels : (415, 6) 
Testing Labels : (83, 6)


In [13]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, 
                    batch_size=batch_size, 
                    epochs=epochs+150, 
                    verbose=1, 
                    validation_data=(X_test, Y_test))

Train on 415 samples, validate on 83 samples
Epoch 1/650
415/415 [==============================] - 3s 8ms/step - loss: 1.2560 - accuracy: 0.5133 - val_loss: 0.9570 - val_accuracy: 0.6627
Epoch 2/650
415/415 [==============================] - 2s 4ms/step - loss: 0.9566 - accuracy: 0.6313 - val_loss: 0.7002 - val_accuracy: 0.7349
Epoch 3/650
415/415 [==============================] - 2s 4ms/step - loss: 0.7723 - accuracy: 0.7422 - val_loss: 0.6632 - val_accuracy: 0.7590
Epoch 4/650
415/415 [==============================] - 2s 4ms/step - loss: 0.7233 - accuracy: 0.7301 - val_loss: 0.5919 - val_accuracy: 0.7590
Epoch 5/650
415/415 [==============================] - 2s 4ms/step - loss: 0.6766 - accuracy: 0.7807 - val_loss: 0.5313 - val_accuracy: 0.8313
Epoch 6/650
415/415 [==============================] - 2s 4ms/step - loss: 0.5970 - accuracy: 0.7855 - val_loss: 0.4876 - val_accuracy: 0.8072
Epoch 7/650
415/415 [==============================] - 2s 4ms/step - loss: 0.5825 - accuracy: 0.7

415/415 [==============================] - 2s 4ms/step - loss: 0.1196 - accuracy: 0.9687 - val_loss: 0.0762 - val_accuracy: 0.9759
Epoch 58/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1573 - accuracy: 0.9446 - val_loss: 0.0601 - val_accuracy: 0.9880
Epoch 59/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1159 - accuracy: 0.9639 - val_loss: 0.0544 - val_accuracy: 0.9880
Epoch 60/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1601 - accuracy: 0.9494 - val_loss: 0.0814 - val_accuracy: 0.9759
Epoch 61/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1562 - accuracy: 0.9518 - val_loss: 0.0567 - val_accuracy: 0.9880
Epoch 62/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1469 - accuracy: 0.9566 - val_loss: 0.0578 - val_accuracy: 0.9759
Epoch 63/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1354 - accuracy: 0.9614 - val_loss: 0.0616 - val_accuracy: 0.9880
Epoch

Epoch 114/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0678 - accuracy: 0.9759 - val_loss: 0.0249 - val_accuracy: 0.9880
Epoch 115/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0683 - accuracy: 0.9807 - val_loss: 0.0694 - val_accuracy: 0.9759
Epoch 116/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1072 - accuracy: 0.9614 - val_loss: 0.0205 - val_accuracy: 0.9880
Epoch 117/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0913 - accuracy: 0.9663 - val_loss: 0.0420 - val_accuracy: 0.9880
Epoch 118/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0750 - accuracy: 0.9735 - val_loss: 0.0236 - val_accuracy: 0.9880
Epoch 119/650
415/415 [==============================] - 2s 4ms/step - loss: 0.1025 - accuracy: 0.9614 - val_loss: 0.0220 - val_accuracy: 0.9880
Epoch 120/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0707 - accuracy: 0.9783 - val_loss: 0.0190 - val_ac

415/415 [==============================] - 2s 4ms/step - loss: 0.0731 - accuracy: 0.9783 - val_loss: 0.0187 - val_accuracy: 0.9880
Epoch 170/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0514 - accuracy: 0.9807 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 171/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0619 - accuracy: 0.9831 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 172/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0727 - accuracy: 0.9807 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 173/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0548 - accuracy: 0.9880 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 174/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0558 - accuracy: 0.9831 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 175/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0586 - accuracy: 0.9783 - val_loss: 0.0035 - val_accuracy: 1.0000

415/415 [==============================] - 2s 4ms/step - loss: 0.0442 - accuracy: 0.9783 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 226/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0533 - accuracy: 0.9855 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 227/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0423 - accuracy: 0.9904 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 228/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0408 - accuracy: 0.9880 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 229/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0448 - accuracy: 0.9880 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 230/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0385 - accuracy: 0.9904 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 231/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0316 - accuracy: 0.9928 - val_loss: 0.0022 - val_accuracy: 1.0000

...

415/415 [==============================] - 2s 4ms/step - loss: 0.0842 - accuracy: 0.9759 - val_loss: 7.8068e-04 - val_accuracy: 1.0000
Epoch 392/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0368 - accuracy: 0.9880 - val_loss: 3.1041e-04 - val_accuracy: 1.0000
Epoch 393/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0263 - accuracy: 0.9928 - val_loss: 3.2410e-04 - val_accuracy: 1.0000
Epoch 394/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0216 - accuracy: 0.9928 - val_loss: 3.0385e-04 - val_accuracy: 1.0000
Epoch 395/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0274 - accuracy: 0.9880 - val_loss: 3.7575e-04 - val_accuracy: 1.0000
Epoch 396/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0172 - accuracy: 0.9928 - val_loss: 4.7992e-04 - val_accuracy: 1.0000
Epoch 397/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0195 - accuracy: 0.9952 - val_loss: 4.213

Epoch 446/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0262 - accuracy: 0.9952 - val_loss: 1.8050e-04 - val_accuracy: 1.0000
Epoch 447/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0188 - accuracy: 0.9952 - val_loss: 1.7589e-04 - val_accuracy: 1.0000
Epoch 448/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0148 - accuracy: 0.9952 - val_loss: 1.7155e-04 - val_accuracy: 1.0000
Epoch 449/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0166 - accuracy: 0.9928 - val_loss: 2.0945e-04 - val_accuracy: 1.0000
Epoch 450/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0159 - accuracy: 0.9904 - val_loss: 1.9615e-04 - val_accuracy: 1.0000
Epoch 451/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0307 - accuracy: 0.9880 - val_loss: 3.5152e-04 - val_accuracy: 1.0000
Epoch 452/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0163 - accuracy: 0.9952 - v

Epoch 501/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0184 - accuracy: 0.9976 - val_loss: 1.1268e-04 - val_accuracy: 1.0000
Epoch 502/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0220 - accuracy: 0.9928 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 503/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0391 - accuracy: 0.9904 - val_loss: 1.0017e-04 - val_accuracy: 1.0000
Epoch 504/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0296 - accuracy: 0.9904 - val_loss: 9.8574e-05 - val_accuracy: 1.0000
Epoch 505/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0124 - accuracy: 0.9976 - val_loss: 1.0534e-04 - val_accuracy: 1.0000
Epoch 506/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0283 - accuracy: 0.9928 - val_loss: 2.3552e-04 - val_accuracy: 1.0000
Epoch 507/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0127 - accuracy: 0.9952 - val_l

Epoch 556/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0167 - accuracy: 0.9928 - val_loss: 7.5816e-05 - val_accuracy: 1.0000
Epoch 557/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 7.4907e-05 - val_accuracy: 1.0000
Epoch 558/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0122 - accuracy: 0.9976 - val_loss: 7.4191e-05 - val_accuracy: 1.0000
Epoch 559/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0133 - accuracy: 0.9952 - val_loss: 8.9331e-05 - val_accuracy: 1.0000
Epoch 560/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0247 - accuracy: 0.9928 - val_loss: 1.0944e-04 - val_accuracy: 1.0000
Epoch 561/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0129 - accuracy: 0.9976 - val_loss: 7.6081e-05 - val_accuracy: 1.0000
Epoch 562/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0252 - accuracy: 0.9904 - v

415/415 [==============================] - 2s 4ms/step - loss: 0.0104 - accuracy: 0.9952 - val_loss: 4.6607e-05 - val_accuracy: 1.0000
Epoch 611/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0083 - accuracy: 0.9976 - val_loss: 4.6399e-05 - val_accuracy: 1.0000
Epoch 612/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0141 - accuracy: 0.9952 - val_loss: 6.6732e-05 - val_accuracy: 1.0000
Epoch 613/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0142 - accuracy: 0.9952 - val_loss: 5.0213e-05 - val_accuracy: 1.0000
Epoch 614/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0136 - accuracy: 0.9952 - val_loss: 7.7574e-05 - val_accuracy: 1.0000
Epoch 615/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 4.2646e-05 - val_accuracy: 1.0000
Epoch 616/650
415/415 [==============================] - 2s 4ms/step - loss: 0.0183 - accuracy: 0.9952 - val_loss: 4.456

# Saving model

In [14]:
# weights = model.get_weights()
# pickle.dump(weights, open(os.path.join("..", "models", "CNN_model.pkl"),"wb"))
# print("Model Saved")

In [17]:
weights = model.get_weights()
# pickle.dump(weights, open(os.path.join("..", "IPD", "CNN_model_half.pkl"),"wb"))
# print("Model Saved")
model_json = model.to_json()
with open(os.path.join("..", "IPD", "CNN_model_json.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join("..", "IPD", "CNN_model_weights.h5"))
# model.save("C:\\Users\\sachi\\Documents\\sem2\\model_final.h5")
print("Saved model to disk")

Saved model to disk
